# PySEAL 

In this notebook, we will explore the [PySEAL library](https://github.com/Huelse/SEAL-Python) - for BGV scheme. 

(NOTE: We have to build this inside the SEAL-python repo. This notebook will not run unless you clone a copy of the [repository](https://github.com/Huelse/SEAL-Python) and then follow the README.md to build first.)

The BGV encryption scheme proposed in 2011 by Brakerski, Gentry and Vaikuntanathan. It is a fully homomorphic encryption scheme that works for both an LWE and an RLWE instance, but they achieve a better performance with the RLWE instance, therefore the focus here will be on the RLWE setup. The way they found for keeping the ciphertext error within a given bound was to use the technique of modulus switching as introduced by Brakerski and Vaikuntanathan. This modulo reduction maps a ciphertext `~c` defined in a ring `Rq`, to a ring `Rp`, where `p < q`, which keeps the error `e` contained within the ciphertext at the same level. This enables one to multiply two ciphertexts and keep the error level constant. By leveraging the bootstrapping technique, one can do this process indefinitely, which opens the way for an FHE scheme.

In [1]:
from seal import *
import numpy as np

In [2]:
# function to print the vector
def print_vector(vector):
    print('[ ', end='')
    for i in range(0, 8):
        print(vector[i], end=', ')
    print('... ]')

In [3]:
# Initialize the encryption parameters for BGV scheme
parms = EncryptionParameters (scheme_type.bgv)
"""
Sets the degree of the polynomial modulus parameter to the specified value.
The polynomial modulus directly affects the number of coefficients in
PlainText polynomials, the size of CipherText elements, the computational
performance of the scheme (bigger is worse), and the security level (bigger
is better).

Specifying the degree of the polynomial modulus parameter.
This modulus is a power of 2 like 1024, 2048, 4096, 8192, 16384, or 32768.
We chose 8192 here because we are taking in account a sample use case where 
poly_modulus_degree is the number of slots per plain text of elements to be 
encoded in single ciphertext in a 2 by n/2 rectangular matrix 
(Type 2^D for D in [10,16])

"""
poly_modulus_degree = 8192
parms.set_poly_modulus_degree(poly_modulus_degree)
parms.set_coeff_modulus(CoeffModulus.BFVDefault(poly_modulus_degree))
parms.set_plain_modulus(PlainModulus.Batching(poly_modulus_degree, 20))
context = SEALContext(parms)

Generate context for BGV scheme

In [4]:
keygen = KeyGenerator(context)
secret_key = keygen.secret_key()
public_key = keygen.create_public_key()
relin_keys = keygen.create_relin_keys()

In [5]:
encryptor = Encryptor(context, public_key)
evaluator = Evaluator(context)
decryptor = Decryptor(context, secret_key)

In [6]:
""" Batch encoder Creates a PlainText from a given matrix. 
This function "batches" a given matrix of either signed or unsigned integers
modulo the PlainText modulus into a PlainText element, and storesthe result 
in the destination parameter.
"""
batch_encoder = BatchEncoder(context)

# The total number of batching slots available to hold data
slot_count = batch_encoder.slot_count()
row_size = slot_count / 2
print(f'Plaintext matrix row size: {row_size}')

Plaintext matrix row size: 4096.0


In [7]:
# Initialize the pod matrix
pod_matrix = [0] * slot_count
pod_matrix[0] = 1
pod_matrix[1] = 2
pod_matrix[2] = 3
pod_matrix[3] = 4

In [8]:
x_plain = batch_encoder.encode(pod_matrix)

#### Let's try and find the noise budget

In [9]:
x_encrypted = encryptor.encrypt(x_plain)
print(f'noise budget in freshly encrypted x: {decryptor.invariant_noise_budget(x_encrypted)}')
print('-'*50)

noise budget in freshly encrypted x: 145
--------------------------------------------------


In [10]:
x_squared = evaluator.square(x_encrypted)
print(f'size of x_squared: {x_squared.size()}')
evaluator.relinearize_inplace(x_squared, relin_keys)
print(f'size of x_squared (after relinearization): {x_squared.size()}')
print(f'noise budget in x_squared: {decryptor.invariant_noise_budget(x_squared)} bits')
decrypted_result = decryptor.decrypt(x_squared)
pod_result = batch_encoder.decode(decrypted_result)
print_vector(pod_result)
print('-'*50)

size of x_squared: 3
size of x_squared (after relinearization): 2
noise budget in x_squared: 109 bits
[ 1, 4, 9, 16, 0, 0, 0, 0, ... ]
--------------------------------------------------


In [11]:
x_4th = evaluator.square(x_squared)
print(f'size of x_4th: {x_4th.size()}')
evaluator.relinearize_inplace(x_4th, relin_keys)
print(f'size of x_4th (after relinearization): { x_4th.size()}')
print(f'noise budget in x_4th: {decryptor.invariant_noise_budget(x_4th)} bits')
decrypted_result = decryptor.decrypt(x_4th)
pod_result = batch_encoder.decode(decrypted_result)
print_vector(pod_result)
print('-'*50)

size of x_4th: 3
size of x_4th (after relinearization): 2
noise budget in x_4th: 34 bits
[ 1, 16, 81, 256, 0, 0, 0, 0, ... ]
--------------------------------------------------


In [12]:
x_8th = evaluator.square(x_4th)
print(f'size of x_8th: {x_8th.size()}')
evaluator.relinearize_inplace(x_8th, relin_keys)
print(f'size of x_8th (after relinearization): { x_8th.size()}')
print(f'noise budget in x_8th: {decryptor.invariant_noise_budget(x_8th)} bits')
decrypted_result = decryptor.decrypt(x_8th)
pod_result = batch_encoder.decode(decrypted_result)
print_vector(pod_result)
print('run out of noise budget')
print('-'*100)

size of x_8th: 3
size of x_8th (after relinearization): 2
noise budget in x_8th: 0 bits
[ -184705, -229604, -73320, -487859, -396130, -483233, 61567, 259176, ... ]
run out of noise budget
----------------------------------------------------------------------------------------------------


#### Now, let's find the noise budget with modulus switching.

In [13]:
x_encrypted = encryptor.encrypt(x_plain)
print(f'noise budget in freshly encrypted x: {decryptor.invariant_noise_budget(x_encrypted)}')
print('-'*50)

noise budget in freshly encrypted x: 145
--------------------------------------------------


In [14]:
x_squared = evaluator.square(x_encrypted)
print(f'size of x_squared: {x_squared.size()}')
evaluator.relinearize_inplace(x_squared, relin_keys)
evaluator.mod_switch_to_next_inplace(x_squared)
print(f'noise budget in x_squared (with modulus switching): {decryptor.invariant_noise_budget(x_squared)} bits')
decrypted_result = decryptor.decrypt(x_squared)
pod_result = batch_encoder.decode(decrypted_result)
print_vector(pod_result)
print('-'*50)

size of x_squared: 3
noise budget in x_squared (with modulus switching): 101 bits
[ 1, 4, 9, 16, 0, 0, 0, 0, ... ]
--------------------------------------------------


In [15]:
x_4th = evaluator.square(x_squared)
print(f'size of x_4th: {x_4th.size()}')
evaluator.relinearize_inplace(x_4th, relin_keys)
evaluator.mod_switch_to_next_inplace(x_4th)
print(f'size of x_4th (after relinearization): { x_4th.size()}')
print(f'noise budget in x_4th (with modulus switching): {decryptor.invariant_noise_budget(x_4th)} bits')
decrypted_result = decryptor.decrypt(x_4th)
pod_result = batch_encoder.decode(decrypted_result)
print_vector(pod_result)
print('-'*50)

size of x_4th: 3
size of x_4th (after relinearization): 2
noise budget in x_4th (with modulus switching): 57 bits
[ 1, 16, 81, 256, 0, 0, 0, 0, ... ]
--------------------------------------------------


In [16]:
x_8th = evaluator.square(x_4th)
print(f'size of x_8th: {x_8th.size()}')
evaluator.relinearize_inplace(x_8th, relin_keys)
evaluator.mod_switch_to_next_inplace(x_8th)
print(f'size of x_8th (after relinearization): { x_8th.size()}')
print(f'noise budget in x_8th (with modulus switching): {decryptor.invariant_noise_budget(x_8th)} bits')
decrypted_result = decryptor.decrypt(x_8th)
pod_result = batch_encoder.decode(decrypted_result)
print_vector(pod_result)

size of x_8th: 3
size of x_8th (after relinearization): 2
noise budget in x_8th (with modulus switching): 14 bits
[ 1, 256, 6561, 65536, 0, 0, 0, 0, ... ]


## Conclusion

In this notebook, we have looked into the BGV scheme using SEAL-python which is a wrapper for [Microsoft SEAL](https://github.com/microsoft/SEAL). Even though Microsoft is great, the limitation for us is that it is written in C++ and the existing wrappers don't do justice to the original work. 

This library is not super convenient to use for operations and lacks documentation for the functinalities. Besides this, it needs to be built inside the repo for SEAL-python, and is not a python library that we could directly import and carry out our operations.